In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [3]:
cdf = spark.read.csv('/dataframe/a_class_info.csv', header=True)
cdf.printSchema()
cdf.show(3)

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
+--------+------+-------------+--------+-----------+-------------+
only showing top 3 rows



### 복잡한 연산
- 일반 select 나 groupby 절만 이용해서는 불가능한 경우 사용
- query 내에 다른 query 를 포함

In [ ]:
cdf.createOrReplaceTempView('class')

#### case 1. 소속된 반의 개수가 2개 이상인 학교들 중 학생 숫자가 가장 적은 학교를 구해보자

In [1]:
# from절 subquery : inline view

# 학생숫자가 가장 적은 학교
spark.sql('select min(class_std_cnt) from class').show(3)

# 학교별로 학생 수의 총 합을 구하고 
spark.sql('select school, sum(class_std_cnt) from class group by school').show(3)

# 학교별로 학생 수의 총 합을 구하고 가장 적은 학생 수를 구하기
spark.sql('''select min(std_tot)
             from (select school, sum(class_std_cnt) as std_tot from class group by school)
             where school is not null
          ''').show(3) # school 값이 null 인 레코드를 제외한 data 에서 school 의 class_std_cnt 합이 가장 적은 수

In [2]:
#case 1. 소속된 반의 개수가 3개 이상인 학교들 중 학생 숫자가 가장 적은 학교를 구해보자
# 학교별로 학생 수의 합, school 값이 null인 경우 제외

spark.sql('''
          select min(std_tot) as min_std_tot
          from(select school, sum(class_std_cnt) as std_tot from class group by school having school is not null and count(school)>=3)
''').show() # 반이 3개 이상인 학교에서 가장 적은 학생 수 - 46


spark.sql('''
          select school 
          from(select school, sum(class_std_cnt) as std_tot from class group by school having school is not null and count(school)>=3)
''').show() # 반이 3개 이상인 학교의 학생수의 총 합과 학교명

# 반의 수가 3개이상 + 학교에서 학생 수 총합이 가장 작은 학교 이름

spark.sql('''
    select school
    from(select school, sum(class_std_cnt) as std_tot from class group by school having school is not null and count(school)>=3)
    where std_tot == (select min(std_tot) as min_std_tot
                      from(select school, sum(class_std_cnt) as std_tot 
                              from class group by school 
                              having school is not null and count(school)>=3))
''').show()

In [ ]:
# df method 사용
cdf.groupby(cdf.school)\
    .agg(sum('class_std_cnt').alias('std_cnt'))\
    .where((count(col('school'))>=3) & (cdf.school.isNotNull()))\
    .select(min('std_cnt')).show()

In [ ]:
cdf.groupby(cdf.school)\
    .agg(sum('class_std_cnt').alias('std_cnt'))\
    .where((count(col('school'))>=3) & (cdf.school.isNotNull()))\
    .where(col('std_cnt') == (cdf.groupby(cdf.school)\
                              .agg(sum('class_std_cnt').alias('std_cnt'))\
                              .where((count(col('school'))>=3) & (cdf.school.isNotNull()))\
                              .select(min('std_cnt')).collect()[0][0])).show()

#### case2. 소속된 반의 개수가 2개 이상인 학교들 중 학생 숫자가 가장 적은 학교보다 학생이 많은 반의 데이터를 구하시오

In [3]:
# # 반별 학생 수의 평균보다 반 학생수가 많은 데이터를 추출하시오.
# spark.sql('''
#     select(*) from class
#     where (class_std_cnt >= avg(class_std_cnt)) and (class_std_cnt is not null)
# ''').show()
# # avg(class_std_cnt) 집계함수가 where 절에서는 사용 불가능

In [4]:
spark.sql('select avg(class_std_cnt) from class') # 반 학생 수의 평균을 구함

In [ ]:
# # 반별 학생 수의 평균보다 반 학생수가 많은 데이터를 추출하시오.
# spark.sql('''
#     select(*) from class
#     where (class_std_cnt >= avg(class_std_cnt)) and (class_std_cnt is not null)
# ''').show()
# # avg(class_std_cnt) 집계함수가 where 절에서는 사용 불가능

In [ ]:
spark.sql('select avg(class_std_cnt) from class').show() # 반 학생 수의 평균을 구함

In [ ]:
spark.sql('''
    select(*)
    from class
    where (class_std_cnt > (select avg(class_std_cnt) from class) and (class_std_cnt is not null))
''').show()

In [ ]:
# cdf.select('*')\
#     .where(cdf.class_std_cnt >= avg(cdf.class_std_cnt))
# where 절에서 집계함수를 사용해서 에러 발생, avg(cdf.class_std_cnt) 코드가 값으로 명시되어야 한다.-> select에서만 사용가능

In [ ]:
type(cdf.select(avg('class_std_cnt')).show()) # df
# Rdd로 변경해주기
cdf.select(avg('class_std_cnt')).collect() # row 객체
cdf.select(avg('class_std_cnt')).collect()[0][0]

In [ ]:
cdf.select('*')\
    .where(cdf.class_std_cnt >= cdf.select(avg('class_std_cnt')).collect()[0][0]).show(3)

### case2. 1. 지역에 따른 학교로 분류하고 분류된 학교의 class_cd 가 2개 초과인 학교별로 반의 학생 수가 가장 작은 반의 학생수를 구하시오

- 위에서 구한 학생수 중 가장 큰 값은?

1. 지역에 따른 학교로 분류 -> 학교의 class_cd 가 2개 초과인 학교들을 추출
2. 추출된 학교들에서 학생 수가 가장 작은 반의 학생 수 추출
3. 2번에서 추출된 학생수들 중 가장 큰 수를 추출

In [5]:
# 참고
# Urban 지역의 학교 vvtva 는 반수가 4개 반 학생 수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school == 'VVTVA') & (cdf.loc == 'Urban')).show()

#### sql

In [6]:
# 지역에 따른 학교로 분류 -> 학교의 class_cd 가 2개 초과인 학교들을 추출 지역에 따른 학교별 반 개수
spark.sql('''
            select loc, school, count(class_cd) as cnt from class 
            group by loc, school 
            having cnt > 2
''').show()

In [ ]:
# 추출된 학교들에서 학생 수가 가장 작은 반의 학생 수 추출 단, school 값이 null 인 데이터 제외
spark.sql('''
            select loc, school, count(class_cd) as cnt, min(class_std_cnt) as min_std from class 
            group by loc, school
            having cnt > 2 and school is not null
''').show()

In [ ]:
# 2번에서 추출된 학생수들 중 가장 큰 수를 추출
spark.sql('''
            select max(min_std)
            from (select loc, school, count(class_cd) as cnt, min(class_std_cnt) as min_std from class 
                  group by loc, school
                  having cnt > 2 and school is not null)
''').show(3)

### df 매서드 활용

In [ ]:
# Urban 지역의 학교 vvtva 는 반수가 4개 반 학생 수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school == 'VVTVA') & (cdf.loc == 'Urban')).show()

In [ ]:
# 지역에 따른 학교로 분류 -> 학교의 class_cd 가 2개 초과인 학교들을 추출 지역에 따른 학교별 반 개수
cdf.groupBy('loc', 'school')\
    .agg(count('class_cd').alias('cnt_cd'))\
    .where((col('cnt_cd') > 2) & (col('school').isNotNull())).show()

In [ ]:
# 추출된 학교들에서 학생 수가 가장 작은 반의 학생 수 추출
cdf.groupBy('loc', 'school')\
    .agg(count('class_cd').alias('cnt_cd'), min('class_std_cnt').alias('min_std'))\
    .where((col('cnt_cd') > 2) & (col('school').isNotNull())).show()

In [ ]:
# 2번에서 추출된 학생수들 중 가장 큰 수를 추출
cdf.groupBy('loc', 'school')\
    .agg(count('class_cd').alias('cnt_cd'), min('class_std_cnt').alias('min_std'))\
    .where((col('cnt_cd') > 2) & (col('school').isNotNull()))\
    .select(max(col('min_std'))).show(30)

#### case 3. 지역에 따른 학교로 분류하고 학교의 class_cd 가 2개 초과인 학교에서 학교별 가장 작은 학생수들을 추출 -> 그 중에서 가장 큰 수를 구하시오

- 구한 수보다 학생 수가 더 많은 반과 학생수를 cdf 전체 데이터에서 추출하시오

#### sql

In [ ]:
spark.sql('''
    select class_cd, class_std_cnt
    from class
    where class_std_cnt > (select max(min_std) as max_minstd
                            from (select loc, school, count(class_cd) as cnt, min(class_std_cnt) as min_std from class 
                            group by loc, school
                            having cnt > 2 and school is not null))
''').show()
spark.sql('select count(*) from class').show()
# Urban 지역의 ZOWMK 학교의 학생수가 가장 작은 반보다 학생 수가 많은 반은 현재 데이터 102개의 class 중 13 클래스이다.

df 매서드

In [ ]:
cdf.select('class_cd', 'class_std_cnt')\
    .where(cdf.class_std_cnt > (cdf.groupBy('loc', 'school')\
                                    .agg(count('class_cd').alias('cnt_cd'), min('class_std_cnt').alias('min_std'))\
                                    .where((col('cnt_cd') > 2) & (col('school').isNotNull()))\
                                    .select(max(col('min_std'))).collect()[0][0])).show()
cdf.count()
# Urban 지역의 ZOWMK 학교의 학생수가 가장 작은 반보다 학생 수가 많은 반은 현재 데이터 102개의 class 중 13 클래스이다.                       

#### case 4. 시골지역의 사립학교 중 표준 교육을 진행하는 학교들의 평균 학생수가 더 많은 도시 지역의 공립학교이면서 특수 교육을 진행하는 학교의 모든 정보를 추출하시오

In [ ]:
cdf.columns
# Loc : Urban, subUrban, Rural(사골)
# school_type : Puble, Non-public
# teaching_type : Standard, Experimental

### sql squery

In [ ]:
# 시골 지역의 사립학교중 표준 교육을 진행하는 학교들의 평균 학생 수
spark.sql('''
    select avg(class_std_cnt)
    from class
    where loc == 'Rural' and school_type == 'Public' and teaching_type == 'Standard'
''').show()

In [7]:
# 평균 학생수가 더 많은 도시 지역의 공립학교이면서 특수 교육을 진행하는 학교의 모든 정보를 추출

spark.sql('''
    select * 
    from class
    where loc == 'Urban' and
          school_type == 'Non-public' and
          teaching_type == 'Experimental' and
          class_std_cnt > (select avg(class_std_cnt)
                            from class
                            where loc == 'Rural' and 
                            school_type == 'Public' and 
                            teaching_type == 'Standard')
''').show()

### df method

#### case 4 sql

In [8]:
# 시골 지역의 사립 학교중 표준 교육을 진행하는 학교들의 평균 수
cdf.groupby(cdf.loc, cdf.school_type, cdf.teaching_type)\
    .agg(avg('class_std_cnt').alias('avg'))\
    .where((cdf.loc == 'Rural') & (cdf.school_type == 'Public') & (cdf.teaching_type == 'Standard'))\
    .select('avg').show()

cdf.groupby(cdf.loc, cdf.school_type, cdf.teaching_type)\
    .agg(avg('class_std_cnt').alias('avg'))\
    .where((cdf.loc == 'Rural') & (cdf.school_type == 'Public') & (cdf.teaching_type == 'Standard'))\
    .select('avg').collect()[0][0]

In [ ]:
# 평균 학생수가 더 많은 도시 지역의 공립학교이면서 특수 교육을 진행하는 학교의 모든 정보를 추출

cdf.select('*')\
    .where((cdf.loc == 'Urban') &
          (cdf.school_type == 'Non-public') &
          (cdf.teaching_type == 'Experimental') &
          (cdf.class_std_cnt > (cdf.groupby(cdf.loc, cdf.school_type, cdf.teaching_type)\
                                  .agg(avg('class_std_cnt').alias('avg'))\
                                  .where((cdf.loc == 'Rural') & (cdf.school_type == 'Public') & (cdf.teaching_type == 'Standard'))\
                                  .select('avg').collect()[0][0]))).show()
